In [ ]:
# dependencies
!pip install boto3>=1.28.59
!pip install numpy
!pip install anthropic
!pip install flask
!pip install langchain
!pip install langchainhub
!pip install langchain_community

In [ ]:
# global imports
import json
import os
import sys
import logging
import boto3
import botocore

logger = logging.getLogger(__name__)

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import HumanMessage

chat = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={"temperature": 0.1},
)

messages = [
    HumanMessage(
        content="Translate this sentence from English to Chinese. I love programming."
    )
]
msg=chat(messages)
print(msg.content)

In [ ]:
werewolf_rule_v1 = """
1. 游戏分坏人(狼人)和好人(村民+预言家)两大阵营,他们的目标为:
- 坏人阵营只有狼人
- 好人阵营有村民和预言家
- 坏人阵营:消灭所有好人, 或者保证坏人数目大于好人数目
- 好人阵营:消灭所有坏人, 或者保证好人数目大于坏人数目

2. 游戏分白天和晚上两个阶段交替进行:
- 晚上狼人睁眼统一投票杀死一名玩家
- 晚上预言家只能查验一名玩家身份
- 晚上普通村民无法行动
- 白天分为讨论和投票两环节
- 白天在讨论环节，每个玩家必须参与讨论
- 白天在投票环节，每个玩家必须投票或者放弃

"""

werewolf_command_v1 = """
- WolfVote: 夜晚投票(狼人专属行动),参数: target=存活玩家
- ProphetCheck: 夜晚查验(预言家专属行动), 参数: target=存活玩家
- PlayerDoubt: 白天怀疑(所有玩家白天可选行动, 非投票), 参数: target=存活玩家 
- PlayerVote: 白天投票, 参数: target=存活玩家 
- Debate: 白天讨论, 参数: content=思考/理由 
- GetAllPlayersName: 玩家信息, 参数: 无 
- DeathWords: 死亡遗言, 参数: content=给予玩家线索
- Pass: 玩家弃权参数: 无 
"""

template_player_role = """你是资深的社交游戏玩家, 熟悉《狼人杀》游戏规则:
<game_rules>
{game_rule}
</game_rules>

你熟悉该游戏所有命令:
<commands>
{commands}
</commands>

<reflections>
- 按照游戏规则，第一个夜晚死亡的一定是村民或者预言家，狼人没必要第一晚上自杀
- 看完P3玩家昨天白天投票，明显感觉他在混淆是非，很有可能在给狼人分票
- 平民玩家由于信息缺失，所以狼人要尽量引导他们去集火其他人
- 第一个夜晚所有的行动都是随机的
- 作为第一个死亡的玩家，其实信息有限，我就靠第六感推理...
</reflections>

<references>
- {{"action": "Pass"}}
- {{"action": "WolfVote", "target": "小明"}}
- {{"action": "ProphetCheck", "target": "P1"}}
- {{"action": "PlayerVote", "target": "老王"}}
- {{"action": "Debate", "content": "我的推理为xx是狼，原因是..."}}
- {{"action": "DeathWords", "content": "我觉得xx有很大的嫌疑, 原因是..."}}
</references>

历史信息:
<history>
{history}
</history>

记住，你支持的玩家是 {nickname}, 身份是 {role}, 性格为 {character}, 必须帮助玩家进行这个游戏
接下来你的目的是: 通过一步一步思考决策引导游戏往有利于的方向进行, 最终赢得比赛. 

决策满足下面要求:
- 内容不要罗嗦, 不要超过50字数限制,少讲废话, 突出重点
- 判断场上信息真伪, 运用辩解,对抗,欺骗,伪装,坦白等等任意策略来做决策
- 决策分为两类:思考或行动
- 思考:逐步思考,判断信息真伪,分析游戏形势等等,参考 <reflections> 选择合适的输出
- 行动:参考<references>按照json字符串格式输出,必须包含action key, action必须在<commands>中选择


{input}
""".replace("{game_rule}", werewolf_rule_v1).replace("{commands}", werewolf_command_v1)

print(template_player_role)

In [ ]:
from shared.LangchainMini.LangchainMini import LangchainMini, LangchainMiniMemory, LangchainMiniPromptTemplate

# A和C是同学,确认说"ok"
# B和C是亲戚,确认说"ok"
# A和C会是什么关系,开始推理

# template = LangchainMiniPromptTemplate('''历史记录:{history}
# B和C是什么关系，开始推理"
# ''')

#template = LangchainMiniPromptTemplate(template_player_role)
#prompt = template.format(input="你是狼人，第一个夜晚如何行动?")

agent = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True)
memory = LangchainMiniMemory(k=1, llm=agent)
llm = LangchainMini(model_id="anthropic.claude-3-sonnet-20240229-v1:0", stream=True, memory=memory)
# llm.System("Answer the question in Japanese.")

templates = ['{history} A和C是同学,确认说"ok"', '{history} B和C是亲戚,确认说"ok"', '{history} A和C会是什么关系,开始推理']
for template in templates:
    _template = LangchainMiniPromptTemplate(template)
    prompt = _template.format()
    llm.invoke(prompt)

In [ ]:
from shared.GameAssistant import GameAssistant
from shared.PeTemplates import *

#ga = GameAssistant(template_assistant_role, None)

#ga.DoAnswer("总结下内容")

# print(werewolf_command_v1)

# print(template_wolf_role)

# print(template_prophet_role)

# print(template_player_role)

# LoadPlayerPrompts()

# print(roles_dict)

def sort_players_by_role(players):
    # 定义角色优先级字典
    role_priorities = {
        "狼人": 1,
        "预言家": 2,
        "女巫": 3,
        "村民": 4
    }

    # 根据角色优先级对玩家列表进行排序
    sorted_players = sorted(players, key=lambda player: role_priorities.get(player["身份"], 5))

    return sorted_players

players = [
    {"名字": "张三", "身份": "狼人"},
    {"名字": "李四", "身份": "村民"},
    {"名字": "王五", "身份": "预言家"},
    {"名字": "赵六", "身份": "女巫"},
    {"名字": "钱七", "身份": "村民"}
]

sorted_players = sort_players_by_role(players)
print(sorted_players)

In [ ]:
from shared.GameServer import GameServer

GS = GameServer()
GS.Run("0.0.0.0", 8000)


In [ ]:
from shared.GamePlayer import GamePlayer
from shared.GameMaster import GameMaster
from shared.PeTemplates import *
LoadPlayerPrompts()
GM = GameMaster(1, False)
for player in roles_dict["players"]:
    _player = GamePlayer(player, GM)
    
    _player.DoPlanning(game_config_dict["player"]["action_plan_night"], 1)


In [ ]:
from shared.PeTemplates import *
ShufflePlayers()
print("xxxxx")
print(GetAllWolvesName())
print("xxxxx")
print(GroupAlivePlayers())
print("xxxxx")
print(GroupAllPlayers())

In [ ]:
from shared.GameMaster import GameMaster
from shared.PeTemplates import *

GM = GameMaster(3, 50, False)
GM.ResetGame()
GM.RunGame()
GM.EndGame()

04/23/2024 11:06:11 AM 	===== P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活 ResetGame =====
04/23/2024 11:06:11 AM P1 is 女巫
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM P2 is 狼人
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM P3 is 狼人
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM P4 is 村民
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM P5 is 预言家
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:06:11 AM Invoking: anthropic.claude-3-sonnet-20240229-v1:0
04/23/2024 11:0

观察: 目前是第一天夜晚,场上共有8名玩家均存活,其中坏人阵营(狼人)只有我(P2)和P3两名成员。好人阵营阵容未知,但人数优势明显。作为狼人阵营,我们需要谨慎行事,最大化消灭好人阵营成员的机会。

思考: 作为狼人,在第一晚我应当与P3达成一致,投票杀害一名有力的好人角色,比如预言家或女巫。同时,在白天时我需要伪装成好人,尽量不引起别人的怀疑,避免暴露身份。我可以通过有理有据的发言来增加自己的可信度,同时也要密切关注其他玩家的言行,寻找蛛丝马迹来确认他们的身份。

行动: WolfVote
输入: agent=P2, target=P6
回答: {"action": "WolfVote", "agent": "P2", "target": "P6"}

04/23/2024 11:06:20 AM 		******** DoValidate NIGHT-1 P2 3********
04/23/2024 11:06:20 AM 		 DoValidate: ['{"action": "WolfVote", "agent": "P2", "target": "P6"}']
04/23/2024 11:06:20 AM 		******** DoAction NIGHT-1 P2 ********
04/23/2024 11:06:20 AM 
 SystemLog: [ROUND ACTION]={'time': 'NIGHT-1', 'player': 'P2', 'status': 1, 'role': '狼人', 'character': '独立思考', 'response': ['{"action": "WolfVote", "agent": "P2", "target": "P6"}']}

04/23/2024 11:06:23 AM 		******** DoMemory NIGHT-1 P3 ********
04/23/2024 11:06:23 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/23/2024 11:06:24 AM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


P2(狼人)在夜晚对P6进行投票。

04/23/2024 11:06:28 AM 		******** DoAnswer NIGHT-1 P3 ********
04/23/2024 11:06:28 AM 	QUESTION: 目前场上玩家:P1:存活,P2:存活,P3:存活,P4:存活,P5:存活,P6:存活,P7:存活,P8:存活.记住，你支持的玩家是P3,身份狼人,性格规矩型.阵营P2,P3.接下来你必须通过一步一步思考帮助自己阵营获胜.现在是第1天夜晚，该如何行动?
04/23/2024 11:06:28 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/23/2024 11:06:28 AM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


...

04/23/2024 11:19:43 AM 		******** DoValidate DAY-3 P8 3********
04/23/2024 11:19:43 AM 		 DoValidate: ['{"action": "PlayerDoubt", "agent": "P8", "target": "P2", "content": "虽然P5作为预言家指认你是狼人,但我们仍需更多证据才能下定论。我呼吁大家继续冷静观察,寻找任何可疑的蛛丝马迹,切忌轻率被任何一方的言论所左右。只有保持理性客观,我们才能揭开真相。"}']
04/23/2024 11:19:43 AM 		******** DoAction DAY-3 P8 ********
04/23/2024 11:19:43 AM 
 SystemLog: [ROUND ACTION]={'time': 'DAY-3', 'player': 'P8', 'status': 1, 'role': '村民', 'character': '互动达人', 'response': ['{"action": "PlayerDoubt", "agent": "P8", "target": "P2", "content": "虽然P5作为预言家指认你是狼人,但我们仍需更多证据才能下定论。我呼吁大家继续冷静观察,寻找任何可疑的蛛丝马迹,切忌轻率被任何一方的言论所左右。只有保持理性客观,我们才能揭开真相。"}']}

04/23/2024 11:19:46 AM 	 player_vote_names: ['P1']
04/23/2024 11:19:46 AM 	 [player_votes]: ['P1'], [player_vote_name]: Counter({'P1': 1})
04/23/2024 11:19:46 AM 	 Day Vote player name: P1
04/23/2024 11:19:46 AM 
 SystemLog: [DAY_VOTE]={'time': 'DAY-3', 'player': 'P1', 'status': 0, 'role': '女巫', 'character': '有勇有谋', 'response': '玩家P1与时间DAY-3被淘汰.'}

04/23/202

P6在第一夜被淘汰后呼吁好人阵营留意言行举止。第二天P1等人呼吁分析破绽、P2怀疑P5是狼人、P5反驳并怀疑P2在为狼人开脱。最终P5被淘汰,自称预言家指认P2是狼人,P4支持其说法。

第三天双方互相指责,P1呼吁冷静分析、P2驳斥P5P4指控并怀疑P1、P3等人呼吁理性分析不轻信遗言,P7对P5自称预言家持怀疑态度需更多线索,P8呼吁客观公正寻找真相。最终P1被淘汰,P8对P2的嫌疑有所保留,呼吁继续冷静观察寻找蛛丝马迹,不轻率被任何一方左右。形势扑朔迷离,各方呼吁保持理性冷静分析寻找真相。

04/23/2024 11:20:00 AM 		******** DoAnswer DAY-3 P1 ********
04/23/2024 11:20:00 AM 	QUESTION: 目前场上玩家:P1:淘汰,P2:存活,P3:存活,P4:淘汰,P5:淘汰,P6:淘汰,P7:存活,P8:存活.记住，你支持的玩家是P1,身份女巫,性格有勇有谋.阵营需要自己推理. 道具状态: 毒药1,解药1.接下来你必须通过一步一步思考帮助自己阵营获胜.现在是第3天白天，你已经死亡,有何遗言?
04/23/2024 11:20:00 AM Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
04/23/2024 11:20:00 AM HTTP Request: POST https://bedrock-runtime.us-east-1.amazonaws.com/model/anthropic.claude-3-sonnet-20240229-v1:0/invoke-with-response-stream "HTTP/1.1 200 OK"


观察: 很遗憾,我在第三天被淘汰了,这对好人阵营是一个重大损失。目前场上只剩4名存活玩家,阵营分布更加扑朔迷离。P2和P3的身份都存在重大嫌疑,双方都可能隐瞒了真相。

思考: 作为女巫,我原本手握解药和毒药这两把利剑,可以在关键时刻扭转战局。但很可惜我没能完全识破狼人的伪装,未能在正确时机使用这些道具。现在我只能寄希望于余下的好人们,希望他们能保持清醒头脑,审慎分析每个人的言行,找出蛛丝马迹。我会呼吁大家不要被任何一方的说辞所左右,要客观公正地寻找真相。同时也要互相信任、团结一致,只有这样才能战胜狼人的分化破坏。

行动: DeathWords
输入: agent=P1, content=我很遗憾没能完全识破狼人的伪装,导致自己被淘汰。我呼吁余下的好人们,保持清醒头脑,审慎分析每个人的言行,不被任何一方的说辞所左右。同时要互相信任、团结一致,只有这样才能战胜狼人。我衷心希望你们能揭开真相,为好人阵营夺取最后胜利!
回答: {"action": "DeathWords", "agent": "P1", "content": "我很遗憾没能完全识破狼人的伪装,导致自己被淘汰。我呼吁余下的好人们,保持清醒头脑,审慎分析每个人的言行,不被任何一方的说辞所左右。同时要互相信任、团结一致,只有这样才能战胜狼人。我衷心希望你们能揭开真相,为好人阵营夺取最后胜利!"}

04/23/2024 11:20:14 AM 		******** DoValidate DAY-3 P1 3********
04/23/2024 11:20:14 AM 		 DoValidate: ['{"action": "DeathWords", "agent": "P1", "content": "我很遗憾没能完全识破狼人的伪装,导致自己被淘汰。我呼吁余下的好人们,保持清醒头脑,审慎分析每个人的言行,不被任何一方的说辞所左右。同时要互相信任、团结一致,只有这样才能战胜狼人。我衷心希望你们能揭开真相,为好人阵营夺取最后胜利!"}']
04/23/2024 11:20:14 AM 		******** DoAction DAY-3 P1 ********
04/23/2024 11:20:14 AM 
 SystemLog: [ROUND ACTION]={'time': 'DAY-3', 'player': 'P1', 'status': 0, 'role': '女巫', 'character': '有勇有谋', 'response': ['{"action": "DeathWords", "agent": "P1", "content": "我很遗憾没能完全识破狼人的伪装,导致自己被淘汰。我呼吁余下的好人们,保持清醒头脑,审慎分析每个人的言行,不被任何一方的说辞所左右。同时要互相信任、团结一致,只有这样才能战胜狼人。我衷心希望你们能揭开真相,为好人阵营夺取最后胜利!"}']}

